## CONTENT BASED FILTERING RECOMMENDATION SYSTEM 

### EUCLIDEAN DISTANCE GENERATOR 

#### LOAD THE LIBRARIES FOR MODEL

In [81]:
from __future__ import print_function
import pandas as pd
import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_columns = 80

#### LOADING THE DATA AND DATA PRELIMINARY CLEANING 

In [82]:
Moviesdf = pd.read_csv ("imdb_processed.csv")
Moviesdf.head()

,Name,Year,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,Dinosaur Planet,2003,tv mini series,"['Documentary', 'Animation', 'Family']",7.7,474.0,['United States'],['English'],"['Christian Slater', 'Scott Sampson']",NaN,NaN,"['Mike Carrol', 'Mike Carroll', 'Georgann Kane']",50.0
1,Get Up and Dance!,1994,video movie,['Family'],8.1,18.0,['United States'],['English'],"['Paula Abdul', 'Aurorah Allain', 'Bill Bohl',...",['Steve Purcell'],NaN,NaN,83.0
2,8 Man,1992,movie,"['Action', 'Sci-Fi']",5.5,93.0,['Japan'],['Japanese'],"['Kai Shishido', 'Etsushi Takahashi', 'Sachiko...",['Yasuhiro Horiuchi'],['Carole King'],"['Kazumasa Hirai', 'Jirô Kuwata', 'Junko Suzuki']",NaN
3,What the #$*! Do We (K)now!?,2004,movie,"['Documentary', 'Comedy', 'Drama', 'Fantasy', ...",5.3,13432.0,['United States'],"['English', 'German', 'Spanish']","['Marlee Matlin', 'Elaine Hendrix', 'John Ross...","['William Arntz', 'Betsy Chasse', 'Mark Vicente']",['Christopher Franke'],"['William Arntz', 'Betsy Chasse', 'Matthew Hof...",60.0
4,Class of Nuke 'Em High Part II: Subhumanoid Me...,1991,movie,"['Comedy', 'Horror', 'Sci-Fi']",4.5,2177.0,['United States'],['English'],"['Brick Bronsky', 'Lisa Gaye', 'Leesa Rowland'...",['Eric Louzil'],['Bob Mithoff'],"['Lloyd Kaufman', 'Carl Morano', 'Matt Unger']",96.0


In [83]:
## Output of the number of rows and columns respectively.
Moviesdf.shape

(8451, 13)

In [84]:
## The information on the dataset
Moviesdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8451 entries, 0 to 8450
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         8451 non-null   object 
 1   Year         8451 non-null   int64  
 2   kind         8451 non-null   object 
 3   genre        8384 non-null   object 
 4   imdb_rating  8135 non-null   float64
 5   vote         8135 non-null   float64
 6   country      8152 non-null   object 
 7   language     8052 non-null   object 
 8   cast         8280 non-null   object 
 9   director     7140 non-null   object 
 10  composer     5722 non-null   object 
 11  writer       7204 non-null   object 
 12  runtime      6798 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 858.4+ KB


In [85]:
## Missing information within the dataframe
Moviesdf.isnull().sum()

Name              0
Year              0
kind              0
genre            67
imdb_rating     316
vote            316
country         299
language        399
cast            171
director       1311
composer       2729
writer         1247
runtime        1653
dtype: int64

In [86]:
# Drop rows with missing values
#Moviesdf = Moviesdf.dropna()
#Moviesdf.isnull().sum()

In [87]:
Moviesdf.shape

(8451, 13)

## CANDIDATE GENERATION 

In [88]:
## Libaries necessary for the function 
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [89]:
### Helping functions for model 
## Splits a DataFrame into training and test sets
def split_dataframe(df, holdout_fraction=0.2):
    total_size = len(df)
    test_size = int(total_size * holdout_fraction)
    
    # Generate a random boolean array to select rows for the test set
    test_indices = np.random.choice(total_size, size=test_size, replace=False)
    
    # Split the DataFrame into training and test sets based on the selected indices
    test = df.iloc[test_indices]
    train = df.drop(df.index[test_indices])
    
    return train, test

In [90]:
## the libraries necessary for the euclidean distance calculation 
# Select relevant columns for similarity calculation (e.g., movie features)
selected_columns = ['kind', 'genre', 'country','language']  # Replace with actual feature names
features = Moviesdf[selected_columns]

In [91]:
# Split the data into training and test sets
training_data, test_data = split_dataframe(Moviesdf, holdout_fraction=0.2)

In [92]:

def euclidean_similarity_generator(training_data, features, k=5):
    # Perform one-hot encoding on categorical features
    training_features = pd.get_dummies(training_data[features])
    
    # Standardize the features
    scaler = StandardScaler()
    training_features = scaler.fit_transform(training_features)
    
    # Calculate pairwise distances between training features
    distances = []
    for i in range(training_features.shape[0]):
        test_movie = training_features[i]
        movie_distances = []
        for j in range(training_features.shape[0]):
            if i != j:  # Exclude the same movie from distance calculation
                training_movie = training_features[j]
                distance = euclidean(test_movie, training_movie)
                movie_distances.append((distance, j))
        distances.append(movie_distances)
    
    # Sort distances and retrieve indices of most similar movies
    similar_movies_indices = []
    for movie_distances in distances:
        movie_distances.sort()
        indices = [index for _, index in movie_distances[:k]]
        similar_movies_indices.append(indices)
        
      # Transform indices to matching movie names
    similar_movies = []
    for indices in similar_movies_indices:
        movie_names = training_data.iloc[indices]['Names'].tolist()
        similar_movies.append(movie_names)   
    
    # Get similar movies' information from training data
    #similar_movies = [training_data.iloc[indices] for indices in similar_movies_indices]
    
    return similar_movies

In [96]:
# Print the similar movies for the first test sample
test_sample_index = 0  # Index of the first test sample
test_sample_name = test_data.iloc[test_sample_index]['Name']  # Name of the first test sample
print(f"Similar movies for test sample '{test_sample_name}':")
print(similar_movie_set.head())
print()

Similar movies for test sample 'The Man Without a Past':
                                       Name  Year            kind  \
899        National Geographic: Inside 9/11  2004  tv mini series   
1467           The Secret Life of the Brain  2002  tv mini series   
1743  Joseph Campbell and the Power of Myth  1988  tv mini series   
2035             Navy SEALs: BUDS Class 234  2000  tv mini series   
2640                  Pregnancy for Dummies  2001  tv mini series   

                genre  imdb_rating    vote            country     language  \
899   ['Documentary']          8.0   489.0  ['United States']  ['English']   
1467  ['Documentary']          7.5    73.0  ['United States']  ['English']   
1743  ['Documentary']          8.9  1029.0  ['United States']  ['English']   
2035  ['Documentary']          8.5   466.0  ['United States']  ['English']   
2640  ['Documentary']          7.7    11.0  ['United States']  ['English']   

                                                   cast dir